In [1]:
import pandas as pd
import pyarrow.dataset as pqds
from pyarrow import fs
import pyarrow as pa
import math
import os

In [2]:
os.getcwd()

'/Users/dexter/sandbox/algo_trading/crypto_feature_preprocess/notebooks'

In [3]:
data_path = "../../crypto_market_data/data/kraken.ETHUSD"
data_set_index = "dsindex"
begin_partition:int = 202201
end_partition:int = 202303
# Read parquet file from data_path as pyarrown dateset
dataset: pqds.dataset = pqds.dataset(
            data_path,
            format="parquet",
            partitioning="hive",
            filesystem=fs.LocalFileSystem(),
        )

criteria = (pqds.field(data_set_index) >= begin_partition) & (
            pqds.field(data_set_index) <= end_partition
        )
dt: pa.table = dataset.to_table(filter=criteria)
df: pd.DataFrame = dt.to_pandas()
#Make column timestamp as pandas datetime index
df.set_index("timestamp", inplace=True)
#convert timestamp ms to datetime
df.index = pd.to_datetime(df.index, unit="ms")
# Resample candlestick to 60 minutes in length
df = df.resample("15Min").agg({"open": "first", "high": "max", "low": "min", "close": "last", "volume": "sum"})
# Remove any rows with NaN values
df.dropna(inplace=True)

In [4]:
df.head()

,open,high,low,close,volume
timestamp,,,,,
2022-01-01 00:00:00,3675.95,3702.03,3675.95,3692.80,217.362166
2022-01-01 00:15:00,3690.98,3698.61,3685.18,3695.60,146.046925
2022-01-01 00:30:00,3696.68,3713.95,3696.01,3713.95,56.123719
2022-01-01 00:45:00,3713.62,3725.00,3713.62,3723.33,138.238059
2022-01-01 01:00:00,3722.79,3725.00,3714.73,3725.00,196.826344


In [12]:
time_inx = df.index[3]
time_inx

Timestamp('2022-01-01 00:45:00')

In [41]:
close, open = df.loc[df.index==time_inx, ["close","open"] ].values[0].tolist()

In [42]:
close

3723.33

In [43]:
open

3713.62

In [5]:
# Shift the close price to the prev row
df["prev_volume"] = df["volume"].shift(1)
df["volume_change"] = df["volume"] / df["prev_volume"]
# calcualte log of price_change column
df["log_volume_change"] = df["volume_change"].apply(lambda x: math.log(x))


In [6]:
from crypto_feature_preprocess.domains.indicators import calculate_log_price_change
log_vol_change = calculate_log_price_change(df["volume"])
df["log_volume_change2"] = log_vol_change


ModuleNotFoundError: No module named 'crypto_feature_preprocess'

In [ ]:
# Check if the log volume is consistent
df = df.iloc[1:]
c = (df["log_volume_change"] - df["log_volume_change2"] < 0.00001)
#show any c = False
df[~c]

In [ ]:
#Plot histogram of log_price_change with 1000 bin and title with plotlib
df["log_volume_change"].hist(bins=100).set_title("log_volume_change")

In [ ]:
# Plot histogram of log_price_change with 1000 bin and fall under 99% percentile and title with plotlib
df["log_volume_change"].hist(bins=100, range=(df["log_volume_change"].quantile(0.0001), df["log_volume_change"].quantile(0.9999))).set_title("log_volume_change")

In [ ]:
from crypto_feature_preprocess.domains.features_gen import Log_Price_Feature
log_vol_feature = Log_Price_Feature(df["volume"], 5, 5)
log_vol_feature_value = log_vol_feature.output_feature_array(normalize=True)

In [ ]:
log_vol_feature_value[:,0]
#Plot the histogram of log_price_feature_value with 1000 bin and fall under 99% percentile and title with plotlib
pd.Series(log_vol_feature_value[:,0]).hist(bins=100, range=(pd.Series(log_vol_feature_value[:,0]).quantile(0.0001), pd.Series(log_vol_feature_value[:,0]).quantile(0.9999))).set_title("log_volume_feature_value")